## Notes:
introduction:<br>
we could easily figure out that fasttext was much more faster than bidreactional lstm or bert when training <br>
and get the high accuracy at the same time!<br>
But the flaw was it could just performce really well on the relatively short sentence,<br>
here are the structure of the model, it was much different than the lstm and rnn but it also use the <br>
wordvec to get better understanding of the text！<br>
## Structure:
parallel trainning make it faster!<br>
like the sentence 200*100(wordvec has 100dim) and sent it to the model,it use 200*1 pooling kernel and we get <br>
vector 100*1 than send it to the linear function and softmax function !


In [30]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for i in n_grams:
        x.append(" ".join(i))
    return x
n = ['i' , 'like' , 'eat' , 'hambuger']
generate_bigrams(n)

['i', 'like', 'eat', 'hambuger', 'eat hambuger', 'like eat', 'i like']

In [31]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  preprocessing = generate_bigrams)

LABEL = data.LabelField(dtype = torch.float)

In [32]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [42]:
print(' '.join(vars(train_data[0])['text']))
print(len(train_data) , ' ' , len(valid_data))

This movie has got to be one of the worst I have ever seen make it to DVD ! ! ! The story line might have clicked if the film had more funding and writers that would have cut the nonsense and sickly scenes that I highly caution parents on .... But the story line is like a loose cannon . If there was such a thing as a drive thru movie maker - this one would have sprung from that . It reminded me a lot of the quickie films that were put out in the 1960 's , poor script writing and filming . < br /><br />The only sensible characters in the whole movie was the bartender and beaver . The rest of the film , could have easily been made by middle school children . I give this film a rating of 1 as it is truly awful and left my entire family with a sense of being cheated . My advice - Don't Watch It ! ! ! I have poor script made by < br of 1 and sickly 's , has got sprung from . My the worst middle school highly caution one would make it a thing as a sensible characters .... But my entire the b

In [43]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [86]:
# see the structure of item after bucketiterator
for i in train_iterator:
    print((vars(i)).keys())
    print((vars(i)))
    print((vars(i))['text'])
    print((vars(i))['text'].shape)
    print(len(vars(train_data[0])['text']))
    break

dict_keys(['batch_size', 'dataset', 'fields', 'input_fields', 'target_fields', 'text', 'label'])
{'batch_size': 64, 'dataset': <torchtext.legacy.data.dataset.Dataset object at 0x000002998800A070>, 'fields': dict_keys(['text', 'label']), 'input_fields': ['text'], 'target_fields': ['label'], 'text': tensor([[   11,    11,   370,  ...,   358,   176,    68],
        [  688,   319,     6,  ...,    52,  9938,    22],
        [    8,    16,  2873,  ...,    98, 10521,    15],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]]), 'label': tensor([1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
        0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 1., 1., 0., 1., 1., 0., 1., 1.])}
tensor([[   11,    11,   370,  ...,   3

In [56]:
import torch.nn as nn 
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self , vocab_size , embedding_dim , output_dim , pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size , embedding_dim , pad_idx)
        
        self.fuc = nn.Linear(embedding_dim , output_dim)
        
    def forward(self , text):
        # sentence shape = [sen lence , batch size]
        embedded = self.embedding(text)
        
        #embedded shape = [sentence len , batch size , embedding len]
        
        embedded = embedded.permute(1,0,2)
        
        #shape = batch_size , sen len , embedding len
        
        pooled = F.avg_pool2d(embedded , (embedded.shape[1] , 1)).squeeze(1)
        
        #shape = batch size , sen embedding len
        return self.fuc(pooled)

In [57]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] #  1

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [61]:
vars(TEXT.vocab).keys()

dict_keys(['freqs', 'itos', 'unk_index', 'stoi', 'vectors'])

In [62]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [63]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.5844, -0.0240, -0.0770,  ..., -1.6428, -0.1380,  0.2442],
        [ 0.9874, -0.3359,  1.8736,  ..., -0.2993, -0.3686,  0.9624],
        [ 0.4455,  0.1342, -2.3334,  ...,  0.9246,  0.3400, -1.0022]])

In [67]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token] #  0

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [71]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [72]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [73]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [74]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [75]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 29s
	Train Loss: 0.686 | Train Acc: 63.33%
	 Val. Loss: 0.628 |  Val. Acc: 70.79%
Epoch: 02 | Epoch Time: 1m 42s
	Train Loss: 0.643 | Train Acc: 73.76%
	 Val. Loss: 0.501 |  Val. Acc: 76.36%
Epoch: 03 | Epoch Time: 1m 37s
	Train Loss: 0.567 | Train Acc: 79.69%
	 Val. Loss: 0.433 |  Val. Acc: 80.54%
Epoch: 04 | Epoch Time: 1m 15s
	Train Loss: 0.490 | Train Acc: 84.34%
	 Val. Loss: 0.389 |  Val. Acc: 83.96%
Epoch: 05 | Epoch Time: 1m 3s
	Train Loss: 0.429 | Train Acc: 87.04%
	 Val. Loss: 0.376 |  Val. Acc: 85.86%


In [76]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

Test Loss: 0.390 | Test Acc: 85.34%


In [77]:
predict_sentiment(model , 'this film is fucking awesome')

0.999998927116394

In [79]:
predict_sentiment(model , 'i feel bad')

1.4799063667234047e-12

In [91]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
input1 = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
print(embedding(input1))
print(embedding(input1).shape)

tensor([[[-1.5810, -0.8412, -0.8202],
         [-1.1554,  0.1725,  0.6804],
         [-0.7392,  0.0038,  1.1208],
         [-0.0988,  0.7145,  0.2685]],

        [[-0.7392,  0.0038,  1.1208],
         [ 0.6734, -0.8284, -0.2839],
         [-1.1554,  0.1725,  0.6804],
         [-0.8718, -0.1155, -1.3543]]], grad_fn=<EmbeddingBackward>)
torch.Size([2, 4, 3])


In [95]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3 ,PAD_IDX)
input1 = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
print(embedding(input1))
print(embedding(input1).shape)

tensor([[[ 0.0000,  0.0000,  0.0000],
         [-1.1740, -2.6945, -0.7820],
         [ 1.2379,  1.9050,  0.1264],
         [ 0.4047, -0.3955, -1.5209]],

        [[ 1.2379,  1.9050,  0.1264],
         [ 0.6784,  0.3445,  1.8218],
         [-1.1740, -2.6945, -0.7820],
         [-1.4771, -0.0763,  0.5093]]], grad_fn=<EmbeddingBackward>)
torch.Size([2, 4, 3])
